In [1]:
using Pkg
Pkg.activate("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\env\\integrate\\")

  Activating project at `C:\Users\Alex\Desktop\repo\2612\0903\dynamical-systems\env\integrate`


In [2]:
using StaticArrays, DifferentialEquations, DynamicalSystems

E, x, y  = 0..40, 0..1,  0..1
box = E × x × y
using CairoMakie, GLMakie

In [3]:
@inbounds U(y, p) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
@inbounds σ(x, p) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
@inbounds g(E, x, y, p, U_) = log( 1.0 + exp( (p[5] * U_ * x * E + p[11]  ) / (p[1]) ) )
@inbounds function TM(u, p, t)
    
    U_ = U(u[3], p)
    
    du1 = (-u[1] + p[1] * g(u[1], u[2], u[3], p, U_) ) / p[2]
    du2 = (1.0 - u[2]) / p[3] - U_*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2], p)
    
    return SVector(du1, du2, du3)
end
@inbounds function jacob_TM_(u, p, t)
    
    U(y, p, exp50) = p[8] + p[9] / ( 1.0 + exp50 )
    U_y(y, p, exp50) = (50.0 * p[9] * exp50) / (1.0 + exp50)^2
    g(E, x, y, p, U_) = exp((p[5]  * U_ * x * E + p[11]) / p[1])
    σ_der(x, p) = exp( (-20.0) * (x - p[6]) )
    exp50 = exp(-50.0 * (u[3] - p[7]))
    
    U_ = U(u[3], p, exp50)
    Uy = U_y(u[3], p, exp50)
    g_ = g(u[1], u[2], u[3], p, U_)
    σ_deri = σ_der(u[2], p)
    
    g_plus = 1.0 + g_
    g_mult = g_ * U_
    g_plus_mult = p[2] * (g_plus)
    u1p5 = p[5] * u[1]
    Uyu2 = Uy * u[2]
    
    E_E = (-1.0 + ((J * u[2] * g_mult)) / (g_plus) ) / p[2]
    E_x = (u1p5 * g_mult) / (g_plus_mult)
    E_y = (u1p5 * Uyu2 * g_) / (g_plus_mult)
    
    x_E = -U_ * u[2]
    x_x = -1.0 / p[3] - U_ * u[1]
    x_y = -Uyu2 * u[1]
    
    y_x = 20.0 * p[10] * σ_deri / (1.0 + σ_deri)^2
    y_y = -1.0/p[4]
    
    SMatrix{3,3}(E_E, x_E, 0.0,
        E_x, x_x, y_x,
        E_y, x_y, y_y)
end
function distance_homoclinic(p1, p2)
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    dist = sqrt( (x1-x2)^2 + (y1-y2)^2 + (z1-z2)^2 )
    return dist
end;

In [4]:
"""
arguments:
- model
- u0
- parameters
- times -- array type of [] which include time_integrate, time_transient
- integ_setting -- settings for method of integration
- t_int_ts -- time inerval for timeseries
- t_int_3d -- time inerval for 3d phase portreit
 - labels -- labels for abscissa and ordinate of timeseries
"""
function research_regime(model, u0, p, times, integ_setting;
        index_3d = [1, 2, 3], index_ts = 1,
        t_int_ts = nothing, t_int_3d = nothing,
        labels = ["time", ""], plot = true)
    
    ds = CoupledODEs(model, u0, p, diffeq = integ_setting)
    tr, trange = trajectory(ds, times[1]; Δt = integ_setting.dt, Ttr = times[2])
    

    t_int_ts = check_t_int(t_int_ts, times)
    t_int_3d = check_t_int(t_int_3d, times)

    inter_3d = interactive_3d(tr, [3, 2, 1], t_int_3d)
    ts = timeseries([trange, tr], index_ts, t_int_ts, labels)
    
    if plot == true
    GLMakie.activate!()
    display(GLMakie.Screen(), inter_3d)
    display(GLMakie.Screen(), ts)
    end
    return [ds, tr, trange, inter_3d, ts]
end
function interactive_3d(data, index, interval)
    f = Figure(resolution = (800, 500))
    axis3 = Axis3(f[1, 1],
                xlabelsize = 35, ylabelsize = 35, zlabelsize = 35,
                xticklabelsize = 20, yticklabelsize = 20, zticklabelsize = 20,
                xgridvisible = false, ygridvisible = false, zgridvisible = false)

    lines!(axis3,
        data[interval[1]:interval[2], index[1]], data[interval[1]:interval[2], index[2]],
        data[interval[1]:interval[2],index[3]], linewidth = 1.5, color = :deeppink)
    return f
end
function check_t_int(t_int, times)
    if t_int == nothing
        t_int = [1, times[1]]
    end
    return t_int
end
function timeseries(data, index, interval, labels)
    
    f = Figure(resolution = (900, 250))
    axis = Axis(f[1, 1], xlabel = labels[1], ylabel = labels[2],
            xlabelsize = 35, ylabelsize = 35,
            xticklabelsize = 28, yticklabelsize = 28,
            xgridvisible = false, ygridvisible = false)
    
    lines!(axis, data[1][interval[1]:interval[2]], data[2][interval[1]:interval[2], index],
            linewidth = 2.5, color = :deeppink)
    return f
end

timeseries (generic function with 1 method)

In [5]:
function research_regime_2_traj(model, u0_1, u0_2, p, times, integ_setting;
        index_3d = [3, 2, 1], t_int_3d = nothing,
        plot = true, onlylastpoint = false, dis_hc = false)
    
    ds_1 = CoupledODEs(model, u0_1, p, diffeq = integ_setting)
    tr_1, trange = trajectory(ds_1, times[1]; Δt = integ_setting.dt, Ttr = times[2])
    
    ds_2 = CoupledODEs(model, u0_2, p, diffeq = integ_setting)
    tr_2, _ = trajectory(ds_2, times[1]; Δt = integ_setting.dt, Ttr = times[2])
    
    fp, eigs, _ = fixedpoints(ds_1, box, jacob_TM_)
    
    if dis_hc == true
        distance_1 = zeros(length(tr_1))
        distance_2 = zeros(length(tr_2))
    
        for i in range(1, length(distance_1), step = 1)
            distance_1[i] = distance_homoclinic(tr_1[i], fp[1])
        end
    
        for i in range(1, length(distance_2), step = 1)
            distance_2[i] = distance_homoclinic(tr_2[i], fp[1])
        end
    
        dis_1 = minimum(distance_1)
        dis_2 = minimum(distance_2)
    end
    
    t_int_3d = check_t_int(t_int_3d, times)

    inter_3d = interactive_3d_2_traj(tr_1, tr_2, index_3d, t_int_3d)
    
    if plot == true
        GLMakie.activate!()
        display(GLMakie.Screen(), inter_3d)
    end
    
    if onlylastpoint == true && dis_hc == true
        return [ds_1, ds_2, tr_1[end], tr_2[end], dis_1, dis_2, inter_3d, fp, eigs]
    elseif onlylastpoint == true
        return [ds_1, ds_2, tr_1[end], tr_2[end], inter_3d]
        else
            return [ds_1, ds_2, tr_1, tr_2, trange, inter_3d]
    end
    
end
function interactive_3d_2_traj(data_1, data_2, index, interval)
    f = Figure(resolution = (800, 500))
    axis3 = Axis3(f[1, 1],
                xlabelsize = 35, ylabelsize = 35, zlabelsize = 35,
                xticklabelsize = 20, yticklabelsize = 20, zticklabelsize = 20,
                xgridvisible = false, ygridvisible = false, zgridvisible = false)

    lines!(axis3,
        data_1[interval[1]:interval[2], index[1]], data_1[interval[1]:interval[2], index[2]],
        data_1[interval[1]:interval[2],index[3]], linewidth = 1.5, color = :deeppink)
    
    lines!(axis3,
        data_2[interval[1]:interval[2], index[1]], data_2[interval[1]:interval[2], index[2]],
        data_2[interval[1]:interval[2],index[3]], linewidth = 1.5, color = :black)
    
    return f
end

interactive_3d_2_traj (generic function with 1 method)

In [6]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const ΔU0 = 0.305; const U0 = 0.264;

In [42]:
time = 3000; tt = 1000; tstep = 0.001
integ_set = (alg = RK4(), adaptive = false, dt = tstep);
times = [time, tt]
time_int_3d = [1, 2500000];

In [50]:
I0 = -1.7099
p = [α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0_1 = output[3] #[3.824643874798959, 0.8282059089413363, 0.45083934164976047]
u0_2 = output[4] #[6.358936446844241, 0.7261925022383418, 0.4505398957896416]
println("last point  pink trajectory:$u0_1")
println("black trajectory:$u0_2")

last point  pink trajectory:[4.079096345692854, 0.851868561809654, 0.46291102118210203]
black trajectory:[9.98134438311113, 0.6873090949163295, 0.4558874981262545]


In [51]:
prev_1 = u0_1
prev_2 = u0_2
println("previous last point  pink trajectory:$prev_1")
println("previous black trajectory:$prev_2")

previous last point  pink trajectory:[4.079096345692854, 0.851868561809654, 0.46291102118210203]
previous black trajectory:[9.98134438311113, 0.6873090949163295, 0.4558874981262545]


In [52]:
output = research_regime_2_traj(TM, u0_1, u0_2, p, times, integ_set,
        t_int_3d = time_int_3d,
        plot = true, onlylastpoint = true, dis_hc = true)

9-element Vector{Any}:
  3-dimensional CoupledODEs
  3-dimensional CoupledODEs
  [1.8438068668735959, 0.9241549645643043, 0.5888327792629277]
  [7.358554604064601, 0.693396039888678, 0.448070557520613]
 0.0013180798334214163
 0.0010418039985404537
  Figure()
  3-dimensional StateSpaceSet{Float64} with 1 points
  Vector{ComplexF64}[[-6.282228490066429 + 0.0im, 2.1497891987933917 - 13.699961989492987im, 2.1497891987933917 + 13.699961989492987im]]

In [53]:
pink = lyapunovspectrum(output[1], time; Ttr = tt)
black = lyapunovspectrum(output[2], time; Ttr = tt)
println("LS  pink trajectory:$pink")
println("LS  black trajectory:$black")

LS  pink trajectory:[0.47080367620043906, 0.00013263763972839595, -2.6964353735275766]
LS  black trajectory:[0.5082761582115884, 0.0001781232180794856, -2.411507185313106]


Double period
[5.703960241685617, 0.7514489808165569, 0.44878444809941526]

Homoclinic
[6.154581240346446, 0.7336262321641008, 0.4403107515803029]

Multistability
U0 = 0.252; I0 = -1.72
[12.769265363312517,0.645078134058814,0.46143383140811733]